In [1]:
using Statistics: mean  # standard library
function loss_and_accuracy(model, data)

    (x,y) = only(loader(data; batchsize=length(data)))
    ŷ = model(x)

    loss = Flux.logitcrossentropy(ŷ, y)  # did not include softmax in the model
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2) # maxa z kolekcji
    (; loss, acc, split=data.split)  # return a NamedTuple
end

loss_and_accuracy (generic function with 1 method)

In [2]:
# Opracowane na podstawie https://github.com/FluxML/model-zoo/blob/3e91af32ebfad628b616618b11bfff2f9f519bec/vision/conv_mnist/conv_mnist.jl
using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x4dim = reshape(data.features, 28, 28, 1, :) # insert trivial channel dim
    yhot  = Flux.onehotbatch(data.targets, 0:9)  # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x4dim, yhot); batchsize, shuffle=true)
end
#=
net = Chain(
    Conv((3, 3), 1 => 6,  relu),
    MaxPool((2, 2)),
    Conv((3, 3), 6 => 16, relu),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(400 => 84, relu), 
    Dense(84 => 10, identity),
)
=#
#= ew. prostsza architektura:
=#
net = Chain(
    Conv((3, 3), 1 => 6,  relu, bias=false),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(13*13*6 => 84, relu, bias=false), 
    Dense(84 => 10, identity, bias=false)
)


Chain(
  Conv((3, 3), 1 => 6, relu, bias=false),  # 54 parameters
  MaxPool((2, 2)),
  Flux.flatten,
  Dense(1014 => 84, relu; bias=false),  # 85_176 parameters
  Dense(84 => 10; bias=false),          # 840 parameters
)                   # Total: 3 arrays, 86_070 parameters, 336.820 KiB.

In [3]:
x1, y1 = first(loader(train_data)); # (28×28×1×1 Array{Float32, 3}, 10×1 OneHotMatrix(::Vector{UInt32}))
y1hat = net(x1)


10×1 Matrix{Float32}:
  0.1354283
 -0.3268636
  0.16558054
  0.010388806
 -0.3624519
  0.13584472
 -0.16289096
 -0.43523616
 -0.06054759
  0.14707138

In [11]:
size(train_data)

(60000,)

In [5]:
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

@show loss_and_accuracy(net, test_data);  # accuracy about 10%, before training

train_log = []
settings = (;
    eta = 1e-2,
    epochs = 1,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net);

hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9)) = [2 4]
loss_and_accuracy(net, test_data) = (loss = 2.3541696f0, acc = 5.99, split = :test)


In [15]:
counter = 0
for epoch in 1:settings.epochs
    @time for (x,y) in loader(train_data, batchsize=settings.batchsize)
        grads = Flux.gradient(model -> Flux.logitcrossentropy(model(x), y), net)
        sizeof(y)
        print(size(x), "\n")
        Flux.update!(opt_state, net, grads[1])
    end
    
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end

(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 100)
(28, 28, 1, 10

┌ Info: 1
│   acc = 94.86
│   test_acc = 94.96
└ @ Main d:\Projects\CNNLibrary\reference_solution.ipynb:12


In [7]:
train_log

Any[]